In [1]:
import pandas as pd
# ================== Preprocessing ====================
# Set the name of the file
rating_filename = 'Ratings.csv'
user_filename = 'Users.csv'
book_filename = 'Books.csv'

ratings_df = pd.read_csv(rating_filename)

print(ratings_df.head())

# Rating - User-ID, ISBN, Book-Rating
# Connect via ISBN

# Age groups
# 0-14 Children
# 15-24 Youth
# 25 - 39 adult1
# 40-64 adult2
# 65+ senior

# error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/

   User-ID        ISBN  Book-Rating
0   276725  034545104X            0
1   276726  0155061224            5
2   276727  0446520802            0
3   276729  052165615X            3
4   276729  0521795028            6


In [2]:
print(len(ratings_df["User-ID"].unique()))
print(len(ratings_df["ISBN"].unique()))
print(ratings_df.shape)

105283
340556
(1149780, 3)


In [3]:
# Take the aggregate ratings (worst case scenario if user is new and age is unknown)
avg_ratings = ratings_df.groupby('ISBN').agg(avg_rating = ('Book-Rating', 'mean'),
                                                number_of_ratings = ('Book-Rating', 'count')).reset_index()

# Keep the books with over 100 ratings
avg_ratings100 = avg_ratings[avg_ratings['number_of_ratings']>100]

In [7]:
# Check popular books
avg_ratings100.sort_values(by='avg_rating', ascending=False).head()

,ISBN,avg_rating,number_of_ratings
97171,0439064864,6.611765,170
97511,0439139597,6.541237,194
97470,0439136350,6.467005,197
144129,0590353403,6.363095,168
98386,043935806X,5.571856,334


In [41]:
# book_ratings_pivot = pd.pivot_table(ratings_df[:1495018980], index='User-ID', columns='ISBN', values='Book-Rating', fill_value = 0)

In [73]:
# Number of ratings not equal for all, should remove 0's
user_counts = ratings_df.groupby('User-ID').agg(number_of_ratings = ('ISBN', 'count')).reset_index()
user_counts.head()

,User-ID,number_of_ratings
0,2,1
1,7,1
2,8,18
3,9,3
4,10,2


In [74]:
# Drop book ratings with 0 because it is already implied and not every book has a rating so it is unnecessary
ratings_df = ratings_df[ratings_df["Book-Rating"] != 0]
ratings_df.shape

(433671, 3)

In [75]:
# Removed a total of 716109 unnecessary rows
print(1149780-433671)

716109


In [69]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

rcLabel, vLabel = ('User-ID', 'ISBN'), 'Book-Rating'
rcCat = [CategoricalDtype(sorted(df[col].unique()), ordered=True) for col in rcLabel]
rc = [df[column].astype(aType).cat.codes for column, aType in zip(rcLabel, rcCat)]
mat = csr_matrix((df[vLabel], rc), shape=tuple(cat.categories.size for cat in rcCat))
dfOut = ( pd.DataFrame.sparse.from_spmatrix(
    mat, index=rcCat[0].categories, columns=rcCat[1].categories) )

In [9]:
import numpy as np
ratings = np.array(ratings_df["Book-Rating"])
users = np.array(ratings_df["User-ID"])
books = np.array(ratings_df["ISBN"])

In [58]:
unique_books = ratings_df["ISBN"].unique()
unique_users = ratings_df["User-ID"].unique()

user_book_map = dict()

In [58]:
x = pd.get_dummies(df.drop

In [59]:
for user in unique_users[:2]:
    books = []
    for book in unique_books:
        if 

array([276725, 276726], dtype=int64)

In [62]:
from surprise import Dataset
from surprise import Reader

reader = Reader(rating_scale=(1,10))

train_df, test_df = df[:len(df)], ratings_df[len(ratings_df)//2:]

data = Dataset.load_from_df(train_df[["User-ID","ISBN","Book-Rating"]],reader)

In [63]:
from surprise import KNNWithMeans

# To use user-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": True,  
}
algo = KNNWithMeans(sim_options=sim_options)

In [64]:
train_set = data.build_full_trainset()
algo.fit(train_set)

Computing the cosine similarity matrix...


MemoryError: Unable to allocate 45.1 GiB for an array with shape (77805, 77805) and data type float64